In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [4]:
# tf should be updated to the latest version 1.8.0

In [5]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import Imputer 
from sklearn.ensemble import RandomForestRegressor 
from sklearn import model_selection
from sklearn import metrics
import textblob
import pickle
from sklearn.externals import joblib
from sklearn.manifold import TSNE

/Users/rithika/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
from scipy.stats import spearmanr, pearsonr

In [22]:
import pandas as pd
import random
import os
import numpy as np
import string
import re
import pickle
import time
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import Imputer 
from sklearn.ensemble import RandomForestRegressor 
from sklearn import model_selection
from sklearn import metrics
from textblob import TextBlob, Word
from sklearn.externals import joblib
from scipy.stats import spearmanr, pearsonr
import gensim
import splitter

In [20]:
#loading googlenews vec
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/rithika/Documents/247ai/GoogleNews-vectors-negative300.bin', binary=True)

# Cleaning data

In [9]:
#os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')

In [10]:
#cleaning input data to get predictor values and target
def prepare_data(fname):
    filename = os.path.join('/Users/rithika/Documents/247ai/datasets', fname)
    if os.path.isfile(filename):
        dat = pd.read_csv(filename)
    else:
        print("no such file exists at this time")
    dat['Adcopy']=dat['Headline 1']+' '+dat['Headline 2']+' '+dat['Description']
    #dat['Adcopy'] = dat['Description']
    dat = dat[['Adcopy','Clicks']]
    dat = dat[pd.notnull(dat['Adcopy'])]
    dat['Adcopy'] = dat['Adcopy'].replace('http\S+|www.\S+', '', regex=True)
    dat['Adcopy'].replace('[™!®"#\'©()*+,-./:;<=>?@\&[\]^_`{|}~’”“′‘\\\%0123456789£]',' ',inplace=True,regex=True)
    dat['Adcopy'].replace('  ',' ',inplace=True,regex=True)
    dat['Adcopy'].replace('   ',' ',inplace=True,regex=True)
    dat['Adcopy'].replace('  ',' ',inplace=True,regex=True)
    dat['Adcopy'] = dat['Adcopy'].str.lower()
    dat['Adcopy'] = dat['Adcopy'].apply(lambda x: ' '.join([wordnet_lemmatizer.lemmatize(str(word)) for word in x.split()]))
    dat['Copy'] = dat['Adcopy'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    data = dat[['Copy','Clicks']]
    datum = data.groupby('Copy')['Clicks'].mean().reset_index()
    print(len(datum))
    return datum

In [12]:
data = prepare_data('Digitonomy_2018-06-01_2018-06-30.csv')

17074


In [13]:
#prints words from the dataset which is not present in google news vec(simply for reducing the runtime of preprocessing)
def no_present(da, model):
    #loading googlenews vec
    #model = gensim.models.KeyedVectors.load_word2vec_format('/Users/rithika/Documents/247ai/GoogleNews-vectors-negative300.bin', binary=True)
    nokey = list()
    for sentence in da:
        for word in sentence.split(' '): 
            if word not in model:
                if word not in nokey:
                    nokey.append(word)
    #print(nokey)
    #print(len(nokey))
    return nokey

In [14]:
#splits the compound words
def spli(row, nokey):
    if row in nokey:
        wo = Word(row).correct() #spellcorrector
        y = splitter.split(row) #compound word splitter
        if y != row and y != '':
            return ' '.join(y)
        else:
            return wo
    else:
        return row

In [15]:
#furthur cleans the data and returns the input and output values
def clean_dat(data,model):
    nokey = no_present(data['Copy'],model)
    data['Copy'] = data['Copy'].apply(lambda x: ' '.join([spli(str(word),nokey) for word in x.split()]))
    datu = data.groupby('Copy')['Clicks'].mean().reset_index()
    clean_d = datu['Copy'] #predictor values
    res = datu['Clicks'] #target
    print(len(clean_d))
    print(len(res))
    return clean_d, res, datu

In [16]:
# removes rows which have words from foreign language
def remove_lang(clean_d, datu, model):
    key_not = no_present(clean_d, model)
    ind = []
    count = 0
    for sen in clean_d:
        for word in sen.split(' '):
            if word in key_not:
                if word not in stop:
                    if count not in ind:
                        ind.append(count)
        count+=1
    cl_data = datu.drop(datu.index[ind])  
    cl_data['Copy'] = cl_data['Copy'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    #print(ind)
    clean_data = cl_data['Copy']
    result = cl_data['Clicks']
    print(len(clean_data))
    print(len(result))
    return clean_data, result

In [23]:
#split and correct
clean_d, res, datu = clean_dat(data,model)

17039
17039


In [24]:
cleaned_data, result = remove_lang(clean_d, datu, model)

16892
16892


In [25]:
#getting unique values 
#datum = data.groupby('Copy')['Clicks'].mean().reset_index()
cleaned_data = cleaned_data.tolist() #predictor values
#result = datum['Clicks'] #target

In [26]:
len(cleaned_data)

16892

In [27]:
len(result)

16892

# Loading pretrained sentence vectors

In [35]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 

In [36]:
#to suppress INFO prints by tensorflow
tf.logging.set_verbosity(tf.logging.ERROR)

In [37]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Sentence vectors

In [38]:
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(cleaned_data))

In [39]:
message_embeddings.shape

(16892, 512)

In [57]:
X = message_embeddings
y = np.array(result)

# Regression with kfold cross validation

In [41]:
#kfold
kf = model_selection.KFold(n_splits=5) 
kf.get_n_splits(X)
print(kf)

KFold(n_splits=5, random_state=None, shuffle=False)


In [21]:
#creating instance of RFRegressor 
model1 = RandomForestRegressor(n_estimators=500,max_features='log2',n_jobs=-1,min_samples_leaf=60)

In [42]:
#cross val split with score and r2
for train_index, test_index in kf.split(X):
    #print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [23]:
#training
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=60, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [24]:
#evaluating
y_pred = model1.predict(X_test)

In [25]:
print('MSE')
print(metrics.mean_squared_error(y_test, y_pred)) #mean square error
print('R2')
print(metrics.r2_score(y_test, y_pred)) #r2 score

MSE
2.961781876033872
R2
-0.005277322313317123


In [26]:
#monotonic relationship as the relation between the variables is non linear
spearman = spearmanr(y_test, y_pred)
print(f'Test data Spearman correlation: {spearman[0]:.3}')

Test data Spearman correlation: 0.0488


In [33]:
#cross val score
#scores = model_selection.cross_val_score(model1, X, y, cv = 5, scoring ='neg_mean_squared_error')
#print(scores)

In [58]:
from sklearn.linear_model import Ridge

In [70]:
def ridge_regression(X, y):
    # Fit the model
    #kfold
    kf = model_selection.KFold(n_splits=5) 
    kf.get_n_splits(X)
    #print(kf)
    #cross val split with score and r2
    for train_index, test_index in kf.split(X):
    #print('TRAIN:', train_index, 'TEST:', test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    model = Ridge(alpha=1, normalize=True, max_iter=1e8)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    spearman = spearmanr(y_test, y_pred)
    pearson = pearsonr(y_test, y_pred)
    print(f'Test data Spearman correlation: {spearman[0]:.3}')
    print(f'Test data pearson correlation: {pearson[0]:.3}')
    print('MSE')
    print(metrics.mean_squared_error(y_test, y_pred)) #mean square error
    print('R2')
    print(metrics.r2_score(y_test, y_pred)) #r2 score
    print('MAE')
    print(metrics.mean_absolute_error(y_test, y_pred)) #mae
    print('Variance Score')
    print(metrics.explained_variance_score(y_test, y_pred)) #mape

In [72]:
ridge_regression(X, y)

Test data Spearman correlation: 0.0201
Test data pearson correlation: 0.0527
MSE
3.0855946080777006
R2
-0.030457168446156313
MAE
0.5084875881335436
Variance Score
-0.02724222611237548


# Results on unseen data

In [34]:
unseen_data = ['get instant decision']
unseen_data1 = ['decision get instant']

In [35]:
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  new_v = session.run(embed(unseen_data))

In [36]:
pred = model1.predict(new_v)

In [37]:
pred.tolist()

[0.7304912344946654]

In [38]:
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  new_v1 = session.run(embed(unseen_data1))

In [39]:
pred1 = model1.predict(new_v1)

In [40]:
pred1.tolist()

[0.836117301641154]